In [1]:
import time
import adodbapi
import pandas as pd
import os
from concurrent.futures import ThreadPoolExecutor, as_completed

In [ ]:
client_manuf =["Malard Nicolas"]
client_brands = ["Nicolas"]
 
decimals = 2
sign = "After"
currency = '€'
currency = ' '+ currency if sign.lower() == 'after' else  currency + ' '
ManufOn=False
prodORitem = "SKU"
categories = ["Effervescents"]
sectors = ["Champagnes","Effervescents Sans Alcool","Mousseux"]
segments = ["Blanc De Blancs","Bruts","Crus","Extra Brut","Millesimes",
            "Rosés","Mousseux Bourgogne","Mousseux Italiens","Mousseux Val De Loire",
            "Mousseux Alsace","Mousseux Bordelais"]
subsegments= []
subcategories= []

national = True
customareas= "REVENUE"
areas = ["NATIONAL","CHANNEL","REGION",f'{customareas}']
 
regions_RET  = []
channels_RET = []
market_RET = []
 
regions_CHAN = ["NICOLAS","HMSM"]
channels_CHAN = ["NICOLAS QCN","NICOLAS VCN","NICOLAS QCT","NICOLAS QCA","NICOLAS CCP"]
market_CHAN = []
 
regions_CUST = []
channels_CUST = ["CA A","CA B","CA C","CA D","CA E"]
market_CUST = []
 
regions_REG =[]
channels_REG = ["NICOLAS IDF","NICOLAS PAC","NICOLAS RHO","NICOLAS AQU","NICOLAS EST"]
market_REG= []
 
 
data_source = "DATA SOURCE: Trade Panel/Retailer Data | Ending June  2025"
years = {2023,2024,2025}
 
 
ManufOrTopC ="Top Companies"
BrandOrTopB = "Top Brands"
end_date = "2025-07-01"
 
past_12_months = pd.date_range(end=end_date, periods=12, freq='ME').strftime('%b-%y').tolist()
National=["NATIONAL"]if national else []
entity_hierarchy = [
    ("Area",National),
    ("RETAILER","Region", regions_RET),
    ("RETAILER","Channel", channels_RET),
    ("RETAILER","Market", market_RET),
    
    ("CHANNEL","Region", regions_CHAN),
    ("CHANNEL","Channel", channels_CHAN),
    ("CHANNEL","Market", market_CHAN),
    
    ("REGION","Region", regions_REG),
    ("REGION","Channel", channels_REG),
    ("REGION","Market", market_REG),
    
    (f"{customareas}","Region", regions_CUST),
    (f"{customareas}","Channel", channels_CUST),
    (f"{customareas}","Market", market_CUST)
]
entity_hierarchy = [
    item for item in entity_hierarchy
    if len(item) == 3 and item[2]  # item[2] is the entity list
]
hierarchy_levels = [
    ("Category", categories),
    ("Sector", sectors),
    ("Segment", segments),
    ("SubSegment", subsegments),
    ("SubCategory", subcategories)
]

direct_parent = {"Sector":"Category",
                "Segment":"Sector",
                "SubSegment":"Segment",
                "SubCategory":"Segment"}
 
server = "powerbi://api.powerbi.com/v1.0/myorg/Nicolas"
dataset_name = "Nicolas Sparkling Dataset"
conn_str = f"Provider=MSOLAP.8;Data Source={server};Initial Catalog={dataset_name};Timeout=600;"
print(conn_str)
 
brackets = ['Base Price Bracket[Base Price Bracket]','Products[Pack Size Bracket]','Products[Pack Count Bracket]']
 
ISDcolumn='Total Size'# default should be 'Total Size'
path=os.path.join(os.getcwd(),"PPA NewEX")
 
# Format months for DAX
p12m_dax = "{" + ", ".join(f'"{date}"' for date in past_12_months) + "}"

Provider=MSOLAP.8;Data Source=powerbi://api.powerbi.com/v1.0/myorg/Nicolas;Initial Catalog=Nicolas Sparkling Dataset;Timeout=600;


In [3]:
start_time = time.time()

##### TopLine By Brand

In [ ]:
def execute_dax_query(BrandorManuf,entity_name, area,market, hierby, filter_p12m, entity_type, bracket):
    outputdic = {}

    time_filter = f"""
        FILTER(
            VALUES('Time Logic'[Time Period]), 
            'Time Logic'[Time Period] = "P12M"
        )
    """ if filter_p12m else ""

    key = f"{entity_type} | {entity_name}"
    filters = ", ".join(filter(None, [time_filter]))

    table_name, column_name = bracket.split("[")
    table_name = table_name.strip()
    column_name = column_name.rstrip("]").strip()

    if table_name == 'Base Price Bracket':
        rowlst = "'Base Price Bracket','Base Price Bracket'[Base Price Bracket]"
        summarize_args = f"'Base Price Bracket'[Base Price Bracket], Products[{BrandorManuf}]"
    else:
        rowlst = f"{table_name},{table_name}[{column_name}]"
        summarize_args = f"'{table_name}'[{column_name}], Products[{BrandorManuf}]"

    if BrandorManuf==f'{BrandOrTopB}':
        # if BrandOrTopB == "Brand":
            columns = ["Value Share", "Brand WoB %", "WoB %", "Value Sales IYA", "Relative Price"]
        # if BrandOrTopB == "SubBrand":
        #     columns = ["Value Share", "SubBrand WoB %", "WoB %", "Value Sales IYA", "Relative Price"]
    else:
        columns = ["Value Share", "Company WoB %", "WoB %", "Value Sales IYA", "Relative Price"]

    
    column_exprs = ", ".join(f'"{col}", COALESCE([{col}], 0)' for col in columns)

    # Main query
    main_dax_query = f"""
        EVALUATE
        CALCULATETABLE(
            ADDCOLUMNS(
                SUMMARIZE(
                    CROSSJOIN(
                        DISTINCT(SELECTCOLUMNS({rowlst})),
                        DISTINCT(SELECTCOLUMNS(Products, Products[{BrandorManuf}]))
                    ),
                    {summarize_args}
                ), 
                {column_exprs}
            ),
            {filters},
            Products[Category] = "{categories[0]}",
            Products[{hierby}] = "{entity_type}",
            FILTER('Market', 'Market'[Area] = "{area}"),
            TREATAS({{"{entity_name}"}}, Market[{market}]),
            FILTER('Scope', 'Scope'[Scope] = "{hierby}")
        )
    """

    # Totals query: uses VALUES to simulate total values per column
    total_dax_query = f"""
        EVALUATE
        CALCULATETABLE(
            ADDCOLUMNS(
                VALUES(Products[{hierby}]),  -- dummy base just to return a single row
                "Value Share", COALESCE([Value Share], 0),
                "Value Sales", COALESCE([Value Sales], 0),
                "Company WoB %", COALESCE([Brand WoB %], 0),  -- Assuming this is Brand WoB %
                "Relative Price", COALESCE([Relative Price], 0),
                "Value Sales IYA", COALESCE([Value Sales IYA], 0)
            ),
            {filters},
            Products[Category] = "{categories[0]}",
            Products[{hierby}] = "{entity_type}",
            FILTER('Market', 'Market'[Area] = "{area}"),
            TREATAS({{"{entity_name}"}}, Market[{market}]),
            FILTER('Scope', 'Scope'[Scope] = "{hierby}")
        )
    """

    try:
        with adodbapi.connect(conn_str) as conn, conn.cursor() as cursor:
            # Execute main query
            cursor.execute(main_dax_query)
            main_columns = [desc[0] for desc in cursor.description]
            main_data = cursor.fetchall()

        with adodbapi.connect(conn_str) as conn, conn.cursor() as cursor:
            # Execute total query
            cursor.execute(total_dax_query)
            total_columns = [desc[0] for desc in cursor.description]
            total_data = cursor.fetchall()
            
        df = pd.DataFrame(main_data, columns=main_columns)
        df.columns = df.columns.str.replace(r'.*\[|\]', '', regex=True)
        df = df.loc[~(df.select_dtypes(include='number') == 0).all(axis=1)]

        if df.empty:
                outputdic[key] = df
                return outputdic
        if not df.empty:
            grouped = []
            numeric_cols = df.select_dtypes(include='number').columns

            for group_name, group_df in df.groupby(df.columns[0]):
                grouped.append(group_df)

                subtotal_values = {
                    col: group_df[col].mean() if "price" in col.lower() else group_df[col].sum()
                    for col in numeric_cols
                }
                subtotal_row = {df.columns[0]: f"{group_name} Total"}
                subtotal_row.update(subtotal_values)
                grouped.append(pd.DataFrame([subtotal_row]))

            df_with_totals = pd.concat(grouped, ignore_index=True)

            # Process grand total row
            if total_data:
                dt = pd.DataFrame([total_data[0]], columns=[col.replace(']', '').split('[')[-1] for col in total_columns])
                grand_total_row = {df.columns[0]: "Grand Total"}
                for col in numeric_cols:
                    if col in dt.columns:
                        grand_total_row[col] = dt[col].values[0]
                df_with_totals = pd.concat([
                    df_with_totals,
                    pd.DataFrame([grand_total_row], columns=df.columns)
                ], ignore_index=True)

        outputdic[key] = df_with_totals if not df.empty else df

    except Exception as e:
        print(f"Error processing {entity_name}: {str(e)}")
        print(f"DAX Query:\n{main_dax_query}")

    return outputdic

def process_dax_queries(BrandorManuf,entity_hierarchy, hierarchy_levels, time_filter, brackets):
    os.makedirs(path, exist_ok=True)  # Ensure output directory exists

    with ThreadPoolExecutor(max_workers=8) as executor:
        for bracket in brackets:
            dfs_results = {}  # Reset dfs_results for each bracket!
            futures = {}
            ordered_keys=[]
            for hierby, hier_values in hierarchy_levels:
                    if isinstance(hier_values, list):
                        for value in hier_values:
                            for area,market, entity_list in entity_hierarchy:
                                for entity in entity_list:
                                    # print(hierby,value,entity)                                    
                                    key = f"{value} | {entity}"
                                    ordered_keys.append(key)
                                    future = executor.submit(execute_dax_query,BrandorManuf, entity, area,market, hierby, time_filter, value, bracket)
                                    futures[future] = key
            temp_results = {}
            for future in as_completed(futures):
                result = future.result()
                temp_results.update(result)

            # Insert results in original order
            for key in ordered_keys:
                if key in temp_results:
                    dfs_results[key] = temp_results[key]
            # Save results only for the current bracket
            column_name = bracket.split("[")[-1].rstrip("]")
            if BrandorManuf==f'{BrandOrTopB}':
                filename = f"share_topline_{column_name}".replace(" ", "_").replace("\xa0", "_").lower()
            else:
                filename = f"share_topline_{column_name}_manuf".replace(" ", "_").replace("\xa0", "_").lower()

            output_file = os.path.join(path, f"{filename}.pkl")

            with open(output_file, "wb") as f:
                pd.to_pickle(dfs_results, f)

            print(f"Saved data for {bracket} to {output_file}.")

# Example function call
process_dax_queries(f'{BrandOrTopB}',entity_hierarchy, hierarchy_levels, time_filter=True, brackets=brackets)
if ManufOn:
    process_dax_queries(f'{ManufOrTopC}',entity_hierarchy, hierarchy_levels, time_filter=True, brackets=brackets)



Saved data for Base Price Bracket[Base Price Bracket] to c:\Users\BW4SA\Documents\Slide-Automate\PPA slide duplicate\PPA NewEX\share_topline_base_price_bracket.pkl.
Saved data for Products[Pack Size Bracket] to c:\Users\BW4SA\Documents\Slide-Automate\PPA slide duplicate\PPA NewEX\share_topline_pack_size_bracket.pkl.
Saved data for Products[Portion Count Bracket] to c:\Users\BW4SA\Documents\Slide-Automate\PPA slide duplicate\PPA NewEX\share_topline_portion_count_bracket.pkl.


In [147]:
if ManufOn==False:
    for bracket in brackets:
        dfs_results = {}  # Reset dfs_results for each bracket!
        futures = {}
        for hierby, hier_values in hierarchy_levels:
            # Save results only for the current bracket
            column_name = bracket.split("[")[-1].rstrip("]")
            filename = f"share_topline_{column_name}_manuf".replace(" ", "_").replace("\xa0", "_").lower()
            output_file = os.path.join(path, f"{filename}.pkl")
            with open(output_file, "wb") as f:
                pd.to_pickle(dfs_results, f)


##### CatScope By Brand

In [ ]:
def execute_dax_query(BrandorMaunf,entity_name, area,market, hierby, filter_p12m, bracket):
    outputdic = {}
    
    time_filter = f"""
        FILTER(
            VALUES('Time Logic'[Time Period]), 
            'Time Logic'[Time Period] = "P12M"
        )
    """ if filter_p12m else ""

    key = f"{entity_name}"
    filters = ", ".join(filter(None, [time_filter]))
    
    table_name, column_name = bracket.split("[")
    table_name = table_name.strip()
    column_name = column_name.rstrip("]").strip()
    
    if table_name == 'Base Price Bracket':
        # Correct format for SUMMARIZE arguments
        rowlst = "'Base Price Bracket','Base Price Bracket'[Base Price Bracket]"
        summarize_args = f"'Base Price Bracket'[Base Price Bracket],Products[{hierby}], Products[{BrandorMaunf}]"
        summarizetotal_args = f"'Base Price Bracket'[Base Price Bracket],Products[{hierby}]"

    else:
        rowlst = f"{table_name},{table_name}[{column_name}]"
        summarize_args = f"'{table_name}'[{column_name}],Products[{hierby}], Products[{BrandorMaunf}]"
        summarizetotal_args = f"'{table_name}'[{column_name}],Products[{hierby}]"
        
    if BrandorMaunf==f'{BrandOrTopB}':
        # if BrandOrTopB == "Brand":
            columns = ["Value Share", "Brand WoB %", "WoB %"]
        # if BrandOrTopB == "SubBrand":
        #     columns = ["Value Share", "SubBrand WoB %", "WoB %"]
    else:
        columns = ["Value Share", 'Company WoB %', "WoB %"]

    column_exprs = ", ".join(f'"{col}", COALESCE([{col}], 0)' for col in columns)

    # Construct the DAX query with correct SUMMARIZE syntax
    main_dax_query = f"""
        EVALUATE
        CALCULATETABLE(
            ADDCOLUMNS(
                SUMMARIZE(
                    CROSSJOIN(
                        DISTINCT(SELECTCOLUMNS({rowlst})),
                        DISTINCT(SELECTCOLUMNS(Products, Products[{hierby}])),
                        DISTINCT(SELECTCOLUMNS(Products, Products[{BrandorMaunf}]))
                    ),
                    {summarize_args}
                ), 
                {column_exprs}
            ),
            {filters},
            Products[Category] = "{categories[0]}",
            FILTER('Market', 'Market'[Area] = "{area}"),
            TREATAS({{"{entity_name}"}}, Market[{market}]),
            FILTER('Scope', 'Scope'[Scope] = "Category")
        )   
    """
    grandtotal_dax_query = f"""
        EVALUATE
        CALCULATETABLE(
            ADDCOLUMNS(
                VALUES(Products[Category]),  -- dummy base just to return a single row
                {column_exprs}

            ),
            {filters},
            Products[Category] = "{categories[0]}",
            FILTER('Market', 'Market'[Area] = "{area}"),
            TREATAS({{"{entity_name}"}}, Market[{market}]),
            FILTER('Scope', 'Scope'[Scope] = "Category")
        )
    """
    maintotal_dax_query = f"""
        EVALUATE
        CALCULATETABLE(
            ADDCOLUMNS(
                SUMMARIZE(
                    CROSSJOIN(
                        DISTINCT(SELECTCOLUMNS({rowlst})),
                        DISTINCT(SELECTCOLUMNS(Products, Products[{hierby}]))
                    ),
                    {summarizetotal_args}
                ), 
                {column_exprs}
            ),
            {filters},
            Products[Category] = "{categories[0]}",
            FILTER('Market', 'Market'[Area] = "{area}"),
            TREATAS({{"{entity_name}"}}, Market[{market}]),
            FILTER('Scope', 'Scope'[Scope] = "Category")
        )
    """

    total_dax_query = f"""
        EVALUATE
        CALCULATETABLE(
            ADDCOLUMNS(
                SUMMARIZE(
                    DISTINCT(SELECTCOLUMNS({rowlst})),
                    '{table_name}'[{column_name}]
                ), 
                {column_exprs}
            ),
            {filters},
            Products[Category] = "{categories[0]}",
            FILTER('Market', 'Market'[Area] = "{area}"),
            TREATAS({{"{entity_name}"}}, Market[{market}]),
            FILTER('Scope', 'Scope'[Scope] = "Category")
        )
    """
    try:
            with adodbapi.connect(conn_str) as conn, conn.cursor() as cursor:
                # Execute main query
                cursor.execute(main_dax_query)
                main_columns = [desc[0] for desc in cursor.description]
                main_data = cursor.fetchall()

            with adodbapi.connect(conn_str) as conn, conn.cursor() as cursor:
                # Execute total query
                cursor.execute(grandtotal_dax_query)
                grandtotal_columns = [desc[0] for desc in cursor.description]
                grandtotal_data = cursor.fetchall()
                
            # Execute main query
            with adodbapi.connect(conn_str) as conn, conn.cursor() as cursor:
                cursor.execute(maintotal_dax_query)
                maintotal_columns = [desc[0] for desc in cursor.description]
                maintotal_data = cursor.fetchall()

            # Execute total query
            with adodbapi.connect(conn_str) as conn, conn.cursor() as cursor:
                cursor.execute(total_dax_query)
                total_columns = [desc[0] for desc in cursor.description]
                total_data = cursor.fetchall()
                
            maintotal_df = pd.DataFrame(maintotal_data, columns=maintotal_columns)
            maintotal_df.columns = maintotal_df.columns.str.replace(r'.*\[|\]', '', regex=True)
            maintotal_df = maintotal_df.loc[~(maintotal_df.select_dtypes(include='number') == 0).all(axis=1)]

            total_df = pd.DataFrame(total_data, columns=total_columns)
            total_df.columns = total_df.columns.str.replace(r'.*\[|\]', '', regex=True)
            total_df = total_df.loc[~(total_df.select_dtypes(include='number') == 0).all(axis=1)]
            if maintotal_df.shape[1] > 1:
                maintotal_df.iloc[:, 1] = maintotal_df.iloc[:, 1].astype(str) + " total"

            # ✅ Prepend "total " to column 0 in total_df
            if total_df.shape[1] > 0:
                total_df.iloc[:, 0] = total_df.iloc[:, 0].astype(str) +" total" 

            if maintotal_df.empty:
                outputdic[key] = maintotal_df
                return outputdic
            if not maintotal_df.empty:
                df_with_totals = pd.concat([maintotal_df, total_df], ignore_index=True)
        
            df = pd.DataFrame(main_data, columns=main_columns)
            df.columns = df.columns.str.replace(r'.*\[|\]', '', regex=True)
            df = df.loc[~(df.select_dtypes(include='number') == 0).all(axis=1)]

            if df.empty:
                    outputdic[key] = df
                    return outputdic
            if not df.empty:
                grouped = []
                numeric_cols = df.select_dtypes(include='number').columns

                # Process grand total row
                if grandtotal_data:
                    dt = pd.DataFrame([grandtotal_data[0]], columns=[col.replace(']', '').split('[')[-1] for col in grandtotal_columns])
                    grand_total_row = {df.columns[0]: "Grand Total"}
                    for col in numeric_cols:
                        if col in dt.columns:
                            grand_total_row[col] = dt[col].values[0]
                    df_with_totals = pd.concat([
                        df,df_with_totals,
                        pd.DataFrame([grand_total_row], columns=df.columns)
                    ], ignore_index=True)

            outputdic[key] = df_with_totals if not df.empty else df

    except Exception as e:
        print(f"Error processing {entity_name}: {str(e)}")
        print(f"DAX Query:\n{main_dax_query}")

    return outputdic

def process_dax_queries(BrandorMaunf,entity_hierarchy, hierarchy_levels, time_filter, brackets):
    os.makedirs(path, exist_ok=True)  # Ensure output directory exists

    with ThreadPoolExecutor(max_workers=8) as executor:
        for bracket in brackets:
            dfs_results = {}  # Reset dfs_results for each bracket!
            futures = {}
            ordered_keys = []

            for hierby, hier_values in hierarchy_levels:
                if hierby=="Category":
                    continue
                for area,market, entity_list in entity_hierarchy:
                    for entity in entity_list:
                        key = f"{entity}"  # Match key used in `execute_dax_query`
                        ordered_keys.append(key)
                        future = executor.submit(execute_dax_query,BrandorMaunf, entity, area,market, hierby, time_filter, bracket)
                        futures[future] = key

                temp_results = {}
                for future in as_completed(futures):
                    result = future.result()
                    temp_results.update(result)

                # Insert results in original order
                for key in ordered_keys:
                    if key in temp_results:
                        dfs_results[key] = temp_results[key]
                # Save results only for the current bracket
                column_name = bracket.split("[")[-1].rstrip("]")
                if BrandorMaunf==f'{BrandOrTopB}':
                    filename = f"share_topline_{column_name}_catscope_{hierby}".replace(" ", "_").replace("\xa0", "_").lower()
                else:
                    filename = f"share_topline_{column_name}_catscope_manuf_{hierby}".replace(" ", "_").replace("\xa0", "_").lower()

                output_file = os.path.join(path, f"{filename}.pkl")

                with open(output_file, "wb") as f:
                    pd.to_pickle(dfs_results, f)

                print(f"Saved data for {bracket} to {output_file}.")

# Example function call
process_dax_queries(f'{BrandOrTopB}',entity_hierarchy, hierarchy_levels, time_filter=True, brackets=brackets)
if ManufOn:
    process_dax_queries(f'{ManufOrTopC}',entity_hierarchy, hierarchy_levels, time_filter=True, brackets=brackets)



Saved data for Base Price Bracket[Base Price Bracket] to c:\Users\BW4SA\Documents\Slide-Automate\PPA slide duplicate\PPA NewEX\share_topline_base_price_bracket_catscope_sector.pkl.
Saved data for Base Price Bracket[Base Price Bracket] to c:\Users\BW4SA\Documents\Slide-Automate\PPA slide duplicate\PPA NewEX\share_topline_base_price_bracket_catscope_segment.pkl.
Saved data for Base Price Bracket[Base Price Bracket] to c:\Users\BW4SA\Documents\Slide-Automate\PPA slide duplicate\PPA NewEX\share_topline_base_price_bracket_catscope_subsegment.pkl.
Saved data for Base Price Bracket[Base Price Bracket] to c:\Users\BW4SA\Documents\Slide-Automate\PPA slide duplicate\PPA NewEX\share_topline_base_price_bracket_catscope_subcategory.pkl.
Saved data for Products[Pack Size Bracket] to c:\Users\BW4SA\Documents\Slide-Automate\PPA slide duplicate\PPA NewEX\share_topline_pack_size_bracket_catscope_sector.pkl.
Saved data for Products[Pack Size Bracket] to c:\Users\BW4SA\Documents\Slide-Automate\PPA slide d

### if ManufOn==False: Create empty  pickle files 


In [149]:
if ManufOn==False:
    for bracket in brackets:
        dfs_results = {}  # Reset dfs_results for each bracket!
        futures = {}
        for hierby, hier_values in hierarchy_levels:
            if hierby=="Category":
                continue
            # Save results only for the current bracket
            column_name = bracket.split("[")[-1].rstrip("]")
            filename = f"share_topline_{column_name}_catscope_manuf_{hierby}".replace(" ", "_").replace("\xa0", "_").lower()
            output_file = os.path.join(path, f"{filename}.pkl")
            with open(output_file, "wb") as f:
                pd.to_pickle(dfs_results, f)


### Parent Scope By Brand

In [ ]:
def execute_dax_query(BrandorMaunf,entity_name, area,market, hierby,direct_parent, filter_p12m, bracket):
    outputdic = {}
    
    time_filter = f"""
        FILTER(
            VALUES('Time Logic'[Time Period]), 
            'Time Logic'[Time Period] = "P12M"
        )
    """ if filter_p12m else ""

    key = f"{entity_name}"
    filters = ", ".join(filter(None, [time_filter]))
    
    table_name, column_name = bracket.split("[")
    table_name = table_name.strip()
    column_name = column_name.rstrip("]").strip()
    
    if table_name == 'Base Price Bracket':
        # Correct format for SUMMARIZE arguments
        rowlst = "'Base Price Bracket','Base Price Bracket'[Base Price Bracket]"
        summarize_args = f"'Base Price Bracket'[Base Price Bracket], Products[{direct_parent[hierby]}],Products[{hierby}], Products[{BrandorMaunf}]"
        summarizeparenttot_args = f"'Base Price Bracket'[Base Price Bracket], Products[{direct_parent[hierby]}]"
        summarizetotal_args = f"'Base Price Bracket'[Base Price Bracket], Products[{direct_parent[hierby]}],Products[{hierby}]"
        

    else:
        rowlst = f"{table_name},{table_name}[{column_name}]"
        summarize_args = f"'{table_name}'[{column_name}], Products[{direct_parent[hierby]}],Products[{hierby}], Products[{BrandorMaunf}]"
        summarizeparenttot_args = f"'{table_name}'[{column_name}], Products[{direct_parent[hierby]}]"
        summarizetotal_args = f"'{table_name}'[{column_name}], Products[{direct_parent[hierby]}],Products[{hierby}]"
        

    if BrandorMaunf==f'{BrandOrTopB}':
        # if BrandOrTopB == "Brand":
            columns = ["Value Share", "Brand WoB %", "WoB %","Value Sales"]
        # if BrandOrTopB == "SubBrand":
        #     columns = ["Value Share", "SubBrand WoB %", "WoB %","Value Sales"]
    else:            
        columns = ["Value Share",'Company WoB %', "WoB %","Value Sales"]

    column_exprs = ", ".join(f'"{col}", COALESCE([{col}], 0)' for col in columns)
    # Construct the DAX query with correct SUMMARIZE syntax
    main_dax_query = f"""
        EVALUATE
        CALCULATETABLE(
            ADDCOLUMNS(
                SUMMARIZE(
                    CROSSJOIN(
                        DISTINCT(SELECTCOLUMNS({rowlst})),
                        DISTINCT(SELECTCOLUMNS(Products, Products[{direct_parent[hierby]}])),
                        DISTINCT(SELECTCOLUMNS(Products, Products[{hierby}])),
                        DISTINCT(SELECTCOLUMNS(Products, Products[{BrandorMaunf}]))
                    ),
                    {summarize_args}
                ), 
                {column_exprs}
            ),
            {filters},
            Products[Category] = "{categories[0]}",
            FILTER('Market', 'Market'[Area] = "{area}"),
            TREATAS({{"{entity_name}"}}, Market[{market}]),
            FILTER('Scope', 'Scope'[Scope] = "{direct_parent[hierby]}")
        )   
    """
    grandtotal_dax_query = f"""
        EVALUATE
        CALCULATETABLE(
            ADDCOLUMNS(
                VALUES(Products[Category]),  -- dummy base just to return a single row
                {column_exprs}

            ),
            {filters},
            Products[Category] = "{categories[0]}",
            FILTER('Market', 'Market'[Area] = "{area}"),
            TREATAS({{"{entity_name}"}}, Market[{market}]),
            FILTER('Scope', 'Scope'[Scope] = "{direct_parent[hierby]}")
        )
    """
    parenttotal_dax_query = f"""
        EVALUATE
        CALCULATETABLE(
            ADDCOLUMNS(
                SUMMARIZE(
                    CROSSJOIN(
                        DISTINCT(SELECTCOLUMNS({rowlst})),
                        DISTINCT(SELECTCOLUMNS(Products, Products[{direct_parent[hierby]}]))
                    ),
                    {summarizeparenttot_args}
                ), 
                {column_exprs}
            ),
            {filters},
            Products[Category] = "{categories[0]}",
            FILTER('Market', 'Market'[Area] = "{area}"),
            TREATAS({{"{entity_name}"}}, Market[{market}]),
            FILTER('Scope', 'Scope'[Scope] = "{direct_parent[hierby]}")
        )
    """
    childtotal_dax_query = f"""
        EVALUATE
        CALCULATETABLE(
            ADDCOLUMNS(
                SUMMARIZE(
                    CROSSJOIN(
                        DISTINCT(SELECTCOLUMNS({rowlst})),
                        DISTINCT(SELECTCOLUMNS(Products, Products[{direct_parent[hierby]}])),
                        DISTINCT(SELECTCOLUMNS(Products, Products[{hierby}]))

                        
                    ),
                    {summarizetotal_args}
                ), 
                {column_exprs}
            ),
            {filters},
            Products[Category] = "{categories[0]}",
            FILTER('Market', 'Market'[Area] = "{area}"),
            TREATAS({{"{entity_name}"}}, Market[{market}]),
            FILTER('Scope', 'Scope'[Scope] = "{direct_parent[hierby]}")
        )
    """

    total_dax_query = f"""
        EVALUATE
        CALCULATETABLE(
            ADDCOLUMNS(
                SUMMARIZE(
                    DISTINCT(SELECTCOLUMNS({rowlst})),
                    '{table_name}'[{column_name}]
                ), 
                {column_exprs}
            ),
            {filters},
            Products[Category] = "{categories[0]}",
            FILTER('Market', 'Market'[Area] = "{area}"),
            TREATAS({{"{entity_name}"}}, Market[{market}]),
            FILTER('Scope', 'Scope'[Scope] = "Category")
        )
    """
    try:
            with adodbapi.connect(conn_str) as conn, conn.cursor() as cursor:
                # Execute main query
                cursor.execute(main_dax_query)
                main_columns = [desc[0] for desc in cursor.description]
                main_data = cursor.fetchall()

            with adodbapi.connect(conn_str) as conn, conn.cursor() as cursor:
                # Execute total query
                cursor.execute(grandtotal_dax_query)
                grandtotal_columns = [desc[0] for desc in cursor.description]
                grandtotal_data = cursor.fetchall()
                
            # Execute main query
            with adodbapi.connect(conn_str) as conn, conn.cursor() as cursor:
                cursor.execute(parenttotal_dax_query)
                maintotal_columns = [desc[0] for desc in cursor.description]
                maintotal_data = cursor.fetchall()
            with adodbapi.connect(conn_str) as conn, conn.cursor() as cursor:
                cursor.execute(childtotal_dax_query)
                childtotal_columns = [desc[0] for desc in cursor.description]
                childtotal_data = cursor.fetchall()

            # Execute total query
            with adodbapi.connect(conn_str) as conn, conn.cursor() as cursor:
                cursor.execute(total_dax_query)
                total_columns = [desc[0] for desc in cursor.description]
                total_data = cursor.fetchall()
                
            maintotal_df = pd.DataFrame(maintotal_data, columns=maintotal_columns)
            maintotal_df.columns = maintotal_df.columns.str.replace(r'.*\[|\]', '', regex=True)
            maintotal_df = maintotal_df.loc[~(maintotal_df.select_dtypes(include='number') == 0).all(axis=1)]
            
            childtotal_df = pd.DataFrame(childtotal_data, columns=childtotal_columns)
            childtotal_df.columns = childtotal_df.columns.str.replace(r'.*\[|\]', '', regex=True)
            childtotal_df = childtotal_df.loc[~(childtotal_df.select_dtypes(include='number') == 0).all(axis=1)]


            total_df = pd.DataFrame(total_data, columns=total_columns)
            total_df.columns = total_df.columns.str.replace(r'.*\[|\]', '', regex=True)
            total_df = total_df.loc[~(total_df.select_dtypes(include='number') == 0).all(axis=1)]
            
            if childtotal_df.shape[1] > 0:
                childtotal_df.iloc[:, 2] = childtotal_df.iloc[:, 2].astype(str) +" total" 
                
            if maintotal_df.shape[1] > 1:
                maintotal_df.iloc[:, 1] = maintotal_df.iloc[:, 1].astype(str) + " total"

            # ✅ Prepend "total " to column 0 in total_df
            if total_df.shape[1] > 0:
                total_df.iloc[:, 0] = total_df.iloc[:, 0].astype(str) +" total" 
                


            if maintotal_df.empty:
                outputdic[key] = maintotal_df
                return outputdic
            if not maintotal_df.empty:
                df_with_totals = pd.concat([maintotal_df,childtotal_df, total_df], ignore_index=True)
        
            df = pd.DataFrame(main_data, columns=main_columns)
            df.columns = df.columns.str.replace(r'.*\[|\]', '', regex=True)
            df = df.loc[~(df.select_dtypes(include='number') == 0).all(axis=1)]

            if df.empty:
                    outputdic[key] = df
                    return outputdic
            if not df.empty:
                grouped = []
                numeric_cols = df.select_dtypes(include='number').columns

                # Process grand total row
                if grandtotal_data:
                    dt = pd.DataFrame([grandtotal_data[0]], columns=[col.replace(']', '').split('[')[-1] for col in grandtotal_columns])
                    grand_total_row = {df.columns[0]: "Grand Total"}
                    for col in numeric_cols:
                        if col in dt.columns:
                            grand_total_row[col] = dt[col].values[0]
                    df_with_totals = pd.concat([
                        df,df_with_totals,
                        pd.DataFrame([grand_total_row], columns=df.columns)
                    ], ignore_index=True)

            outputdic[key] = df_with_totals if not df.empty else df

    except Exception as e:
        print(f"Error processing {entity_name}: {str(e)}")
        print(f"DAX Query:\n{main_dax_query}")

    return outputdic


def process_dax_queries(BrandorMaunf,entity_hierarchy, hierarchy_levels, direct_parent, time_filter, brackets):
    os.makedirs(path, exist_ok=True)  # Ensure output directory exists

    with ThreadPoolExecutor(max_workers=8) as executor:
        for bracket in brackets:
            for hierby, hier_values in hierarchy_levels:
                if hierby == "Category":
                    continue

                print(f"Processing bracket: {bracket} | hierby: {hierby}")
                dfs_results = {}  # Reset for each combination of bracket + hierby
                futures = {}

                for area,market, entity_list in entity_hierarchy:
                    for entity in entity_list:
                        future = executor.submit(execute_dax_query,BrandorMaunf, entity, area,market, hierby, direct_parent, time_filter, bracket)
                        futures[future] = (entity, area)

                for future in as_completed(futures):
                    result = future.result()
                    dfs_results.update(result)

                column_name = bracket.split("[")[-1].rstrip("]")
                if BrandorMaunf==f"{ManufOrTopC}":
                    filename = f"share_topline_{column_name}_parentscope_manuf_{hierby}".replace(" ", "_").replace("\xa0", "_").lower()
                else:    
                    filename = f"share_topline_{column_name}_parentscope_{hierby}".replace(" ", "_").replace("\xa0", "_").lower()
                output_file = os.path.join(path, f"{filename}.pkl")

                with open(output_file, "wb") as f:
                    pd.to_pickle(dfs_results, f)

                print(f"✅ Saved data for bracket: {bracket}, hierby: {hierby} to {output_file}")


# Example function call
process_dax_queries(f"{BrandOrTopB}",entity_hierarchy, hierarchy_levels,direct_parent, time_filter=True, brackets=brackets)
if ManufOn:
    process_dax_queries(f"{ManufOrTopC}",entity_hierarchy, hierarchy_levels,direct_parent, time_filter=True, brackets=brackets)



Processing bracket: Base Price Bracket[Base Price Bracket] | hierby: Sector
✅ Saved data for bracket: Base Price Bracket[Base Price Bracket], hierby: Sector to c:\Users\BW4SA\Documents\Slide-Automate\PPA slide duplicate\PPA NewEX\share_topline_base_price_bracket_parentscope_sector.pkl
Processing bracket: Base Price Bracket[Base Price Bracket] | hierby: Segment
✅ Saved data for bracket: Base Price Bracket[Base Price Bracket], hierby: Segment to c:\Users\BW4SA\Documents\Slide-Automate\PPA slide duplicate\PPA NewEX\share_topline_base_price_bracket_parentscope_segment.pkl
Processing bracket: Base Price Bracket[Base Price Bracket] | hierby: SubSegment
✅ Saved data for bracket: Base Price Bracket[Base Price Bracket], hierby: SubSegment to c:\Users\BW4SA\Documents\Slide-Automate\PPA slide duplicate\PPA NewEX\share_topline_base_price_bracket_parentscope_subsegment.pkl
Processing bracket: Base Price Bracket[Base Price Bracket] | hierby: SubCategory
✅ Saved data for bracket: Base Price Bracket[B

### if ManufOn==False: Create empty  pickle files 

In [151]:
if ManufOn==False:
    for bracket in brackets:
        dfs_results = {}  # Reset dfs_results for each bracket!
        futures = {}
        for hierby, hier_values in hierarchy_levels:
            if hierby=="Category" or direct_parent[hierby]=="Category":
                continue
            # Save results only for the current bracket
            column_name = bracket.split("[")[-1].rstrip("]")
            filename = f"share_topline_{column_name}_parentscope_manuf_{hierby}".replace(" ", "_").replace("\xa0", "_").lower()
            output_file = os.path.join(path, f"{filename}.pkl")
            with open(output_file, "wb") as f:
                pd.to_pickle(dfs_results, f)


## Inter-size Discount

In [ ]:
def execute_dax_query(entity_name, area,market, filter_p3m, brand=None):
    outputdic = {}
    
    time_filter = """
        FILTER(
            VALUES('Time Logic'[Time Period]), 
            'Time Logic'[Time Period] = "P3M"
        )
    """ if filter_p3m else ""

    brand_filter = f'Products[{BrandOrTopB}]="{brand}"' if brand else ""
    key = f"{categories[0]} | {brand} | {entity_name}" if brand else ""
    filters = ", ".join(filter(None, [brand_filter, time_filter]))

    # Define columns dynamically
    columns = [
        "Base Price/Unit","Value Sales", "Unit Sales","Value Sales IYA","VSOD"
    ]
    column_exprs = ", ".join(f'"{col}", COALESCE([{col}], 0)' for col in columns)
    row_list=[]
    if ISDcolumn != 'Total Size':
        her_list = ["Products[Variant]", "Products[Total Size]", f"Products[{ISDcolumn}]"]
    else:
        her_list = ["Products[Variant]", "Products[Total Size]"]
    if her_list:
        summarize_expr = f"SUMMARIZE(Products, {', '.join(her_list)})"
        
    main_dax_query = f"""
        EVALUATE
        CALCULATETABLE(
            ADDCOLUMNS(
                {summarize_expr},
                {column_exprs}
            ),
            {filters},
            Products[Category] = "{categories[0]}",     
            FILTER('Market', 'Market'[Area] = "{area}"),       
            TREATAS({{"{entity_name}"}}, Market[{market}]),
            FILTER('Scope', 'Scope'[Scope] = "Category")
        )   
    """
    total_dax_query = f"""
        EVALUATE
        CALCULATETABLE(
            ADDCOLUMNS(
                VALUES(Products[Category]),
                {column_exprs}
            ),
            {filters},
            Products[Category] = "{categories[0]}",
            FILTER('Market', 'Market'[Area] = "{area}"),
            TREATAS({{"{entity_name}"}}, Market[{market}]),
            FILTER('Scope', 'Scope'[Scope] = "Category")
        )   
    """
    try:
        with adodbapi.connect(conn_str) as conn, conn.cursor() as cursor:
            # Execute main query
            cursor.execute(main_dax_query)
            main_columns = [desc[0] for desc in cursor.description]
            main_data = cursor.fetchall()

        with adodbapi.connect(conn_str) as conn, conn.cursor() as cursor:
            # Execute total query
            cursor.execute(total_dax_query)
            total_columns = [desc[0] for desc in cursor.description]
            total_data = cursor.fetchall()
            
        df = pd.DataFrame(main_data, columns=main_columns)
        df.columns = df.columns.str.replace(r'.*\[|\]', '', regex=True)
        df = df.loc[~(df.select_dtypes(include='number') == 0).all(axis=1)]

        if df.empty:
                outputdic[key] = df
                return outputdic
        if not df.empty:
            grouped = []
            numeric_cols = df.select_dtypes(include='number').columns

            for group_name, group_df in df.groupby(df.columns[0]):
                grouped.append(group_df)

                subtotal_values = {
                    col: group_df[col].mean() if "price" in col.lower() else group_df[col].sum()
                    for col in numeric_cols
                }
                subtotal_row = {df.columns[0]: f"{group_name} Total"}
                subtotal_row.update(subtotal_values)
                grouped.append(pd.DataFrame([subtotal_row]))

            df_with_totals = pd.concat(grouped, ignore_index=True)

            # Process grand total row
            if total_data:
                dt = pd.DataFrame([total_data[0]], columns=[col.replace(']', '').split('[')[-1] for col in total_columns])
                grand_total_row = {df.columns[0]: "Grand Total"}
                for col in numeric_cols:
                    if col in dt.columns:
                        grand_total_row[col] = dt[col].values[0]
                df_with_totals = pd.concat([
                    df_with_totals,
                    pd.DataFrame([grand_total_row], columns=df.columns)
                ], ignore_index=True)

        outputdic[key] = df_with_totals if not df.empty else df

    except Exception as e:
        print(f"Error processing {entity_name}: {str(e)}")
        print(f"DAX Query:\n{main_dax_query}")

    return outputdic



def process_dax_queries(entity_hierarchy, time_filter, client_brands=None):
    dfs_results = {}
    with ThreadPoolExecutor(max_workers=8) as executor:
        futures = []
        for area,market, entity_list in entity_hierarchy:
            for entity in entity_list:                    
                if client_brands:
                    for brand in client_brands:
                        futures.append(executor.submit(execute_dax_query, entity, area,market, time_filter, brand=brand))

        for future in as_completed(futures):
            try:
                result = future.result()
                dfs_results.update(result)
            except Exception as e:
                print(f"Error processing query: {e}")

        # Construct the output file name correctly
        
        filename = "inter_size.pkl" 

        output_file = f"{path}\\{filename}"

        with open(output_file, "wb") as f:
            pd.to_pickle(dfs_results, f)

        print(f"All DataFrames saved to {output_file}.")


process_dax_queries(entity_hierarchy, time_filter=True,client_brands=client_brands)  # Brand-level

All DataFrames saved to c:\Users\BW4SA\Documents\Slide-Automate\PPA slide duplicate\PPA NewEX\inter_size.pkl.


## if ManufOn==False: Create empty  pickle files 

In [153]:
if ManufOn==False:
    for bracket in brackets:
        dfs_results = {}  # Reset dfs_results for each bracket!
        column_name = bracket.split("[")[-1].rstrip("]")
        filename = f"share_topline_{column_name}_manuf".replace(" ", "_").replace("\xa0", "_").lower()
        output_file = os.path.join(path, f"{filename}.pkl")
        with open(output_file, "wb") as f:
            pd.to_pickle(dfs_results, f)


# Extraction  NewSlide BY RET

####### Brand

In [ ]:
def execute_dax_query(BrandorMaunf, entity_name, hierby, filter_p12m, area, bracket, entity_type=None, row_field=None,areamarket=True):
    outputdic = {}

    time_filter = """
        FILTER(
            VALUES('Time Logic'[Time Period]), 
            'Time Logic'[Time Period] = "P12M"
        )
    """ if filter_p12m else ""

    key = f"{entity_name}"

    crossjoin_parts = []

    if isinstance(row_field, list):
        for field in row_field:
            table, col = field.split("[")
            col = col.rstrip("]")
            crossjoin_parts.append(
                f'DISTINCT(SELECTCOLUMNS({table.strip()}, "{col.strip()}", {table.strip()}[{col.strip()}]))'
            )
    else:
        table, col = row_field.split("[")
        col = col.rstrip("]")
        crossjoin_parts.append(
            f'DISTINCT(SELECTCOLUMNS({table.strip()}, "{col.strip()}", {table.strip()}[{col.strip()}]))'
        )

    crossjoin_expr = ",\n                ".join(crossjoin_parts)
    row_fields_expr = ", ".join(
    f"[{field.split('[')[1].rstrip(']')}]"
    for field in row_field
    ) if isinstance(row_field, list) else f"[{row_field.split('[')[1].rstrip(']')}]"

    # print(row_fields_expr)
    table_name, column_name = bracket.split("[")
    table_name = table_name.strip()
    column_name = column_name.rstrip("]").strip()

    # Build summarize_args as a string with proper quoting
    if table_name == 'Base Price Bracket':
        column_name="Base Price Bracket"

    else:
        rowlst = f"{table_name},{table_name}[{column_name}]"

        
    # Define columns for ADDCOLUMNS dynamicall
    # print(crossjoin_expr)
    crossjoin_expr_list = [crossjoin_expr]
    crossjoin_expr_list=crossjoin_expr_list[0]
    row_fields_exprlist=[row_fields_expr]
    row_fields_expr_list = [field.strip() for field in row_fields_expr.split(",")]
    row_fields_expr_first = row_fields_expr_list[0]
    if areamarket is True and area:
        areafilter = f"""FILTER('Market', 'Market'[Area] = "{area}"),"""
    else:
        areafilter = ""


    areafilter = ", ".join(filter(None, [areafilter]))

    dax_query = f"""
    EVALUATE
    CALCULATETABLE(
        ADDCOLUMNS(
            SUMMARIZE(
                CROSSJOIN(

                    DISTINCT(SELECTCOLUMNS('{table_name}', "{column_name}", '{table_name}'[{column_name}])),
                    DISTINCT(SELECTCOLUMNS(Products, "{BrandorMaunf}", Products[{BrandorMaunf}])),
                    {crossjoin_expr}
                    
                ),
                [{column_name}], [{BrandorMaunf}],{row_fields_expr}
            ),
            "Value Share", COALESCE([Value Share], 0),
            "WoB %", COALESCE([WoB %], 0),
            "Value Sales IYA", COALESCE([Value Sales IYA], 0),
            "Relative Price", COALESCE([Relative Price], 0)
        ),
        {time_filter},
        {areafilter}
        FILTER(Products, Products[Category] = "{categories[0]}" && Products[{hierby}] = "{entity_type}"),
        FILTER('Scope', 'Scope'[Scope] = "{hierby}")
    )
    """
    
    maintotal_dax_query = f"""
        EVALUATE
            CALCULATETABLE(
                ADDCOLUMNS(
                    SUMMARIZE(
                    CROSSJOIN(
                        DISTINCT(SELECTCOLUMNS('{table_name}', "{column_name}", '{table_name}'[{column_name}])),
                        DISTINCT(SELECTCOLUMNS(Products, "{BrandorMaunf}", Products[{BrandorMaunf}])),
                        
                        {crossjoin_expr_list}
                        
                    ),
                    [{column_name}], [{BrandorMaunf}],{row_fields_expr_first}
                ),
                    "Value Share", COALESCE([Value Share], 0),
                    "WoB %", COALESCE([WoB %], 0),
                    "Value Sales IYA", COALESCE([Value Sales IYA], 0),
                    "Relative Price", COALESCE([Relative Price], 0)
                ),
                {time_filter},
                {areafilter}       
                FILTER(Products, Products[Category] = "{categories[0]}" && Products[{hierby}] = "{entity_type}"),
                FILTER('Scope', 'Scope'[Scope] = "{hierby}")
            )
    """

    try:
        with adodbapi.connect(conn_str) as conn, conn.cursor() as cursor:
            cursor.execute(dax_query)
            columns = [desc[0] for desc in cursor.description]
            data = cursor.fetchall()
            
        with adodbapi.connect(conn_str) as conn, conn.cursor() as cursor:
            cursor.execute(maintotal_dax_query)
            maintotal_columns = [desc[0] for desc in cursor.description]
            maintotal_data = cursor.fetchall()

        df = pd.DataFrame(data, columns=columns)
        df.columns = df.columns.str.replace(r'.*\[|\]', '', regex=True)
        df = df.loc[~(df.select_dtypes(include='number') == 0).all(axis=1)]

        maintotal_df = pd.DataFrame(maintotal_data, columns=maintotal_columns)
        maintotal_df.columns = maintotal_df.columns.str.replace(r'.*\[|\]', '', regex=True)
        
        maintotal_df = maintotal_df.loc[~(maintotal_df.select_dtypes(include='number') == 0).all(axis=1)]
        if maintotal_df.shape[1] > 1:
                maintotal_df.iloc[:, 2] = maintotal_df.iloc[:, 2].astype(str) + " total"


        if not maintotal_df.empty:
            # Ensure the total dataframe has the same columns as df
            for col in df.columns:
                if col not in maintotal_df.columns:
                    maintotal_df[col] = pd.NA  # or np.nan if you prefer numeric NaNs

            # Reorder columns to match df
            maintotal_df = maintotal_df[df.columns]

            # Concatenate totals on top of the main dataframe
            df_with_totals = pd.concat([maintotal_df, df], ignore_index=True)


            outputdic[key] = df_with_totals
        print(f"Query executed successfully for {hierby} | {row_field}.")
    except adodbapi.DatabaseError as db_error:
        print(f"Database error for {hierby} {entity_type} | {row_field}: {db_error}")

    return outputdic


def process_dax_queries(BrandorManuf, hierarchy_levels, time_filter, area, row_list, brackets,areamarket):
    with ThreadPoolExecutor(max_workers=8) as executor:
        for bracket in brackets:
            dfs_results = {}  # ← Reset per bracket
            futures = []      # ← Reset per bracket

            for hierby, hier_values in hierarchy_levels:
                for value in hier_values:
                    futures.append(executor.submit(
                        execute_dax_query,
                        BrandorManuf, value, hierby, time_filter, area, bracket,
                        entity_type=value, row_field=row_list
                    ))

            for future in as_completed(futures):
                try:
                    result = future.result()
                    dfs_results.update(result)
                except Exception as e:
                    print(f"Error processing query: {e}")

            # suffix = row_list[0].split('[')[-1].rstrip(']')
            column_name = bracket.split("[")[-1].rstrip("]")
            if area == "RETAILER" and areamarket==True :
                filename = f"modified_retailer_{column_name}.pkl".replace(" ", "_").replace("\xa0", "_").lower()
            elif area == "CHANNEL" and areamarket==True:
                filename = f"modified_channels_{column_name}.pkl".replace(" ", "_").replace("\xa0", "_").lower()
            elif area == "REGION" and areamarket==True:
                filename = f"modified_region_{column_name}.pkl".replace(" ", "_").replace("\xa0", "_").lower()
            elif area == customareas and areamarket==True:
                filename = f"modified_cust_{column_name}.pkl".replace(" ", "_").replace("\xa0", "_").lower()
            else:
                filename = f"modified_Area_{column_name}.pkl".replace(" ", "_").replace("\xa0", "_").lower()

            output_file = f"{path}\\{filename}"
            with open(output_file, "wb") as f:
                pd.to_pickle(dfs_results, f)
            print(f"Saved DataFrames for {row_list} to {output_file}.")


area_list = []
area_list.append('Market[Area]')
process_dax_queries(BrandOrTopB, hierarchy_levels, time_filter=True, area="", row_list=area_list, brackets=brackets,areamarket=False)

# Execute per area
if "RETAILER" in areas:
    RET_list = []
    if regions_RET:
        RET_list.append('Market[Region]')
    if channels_RET:
        RET_list.append('Market[Channel]')
    if market_RET:
        RET_list.append('Market[Market]')
    process_dax_queries(BrandOrTopB, hierarchy_levels, time_filter=True, area="RETAILER", row_list=RET_list, brackets=brackets,areamarket=True)

if "CHANNEL" in areas:
    CHA_list = []
    if regions_CHAN:
        CHA_list.append('Market[Region]')
    if channels_CHAN:
        CHA_list.append('Market[Channel]')
    if market_CHAN:
        CHA_list.append('Market[Market]')
    process_dax_queries(BrandOrTopB, hierarchy_levels, time_filter=True, area="CHANNEL", row_list=CHA_list, brackets=brackets,areamarket=True)

if "REGION" in areas:
    REG_list = []
    if regions_CHAN:
        REG_list.append('Market[Region]')
    if channels_CHAN:
        REG_list.append('Market[Channel]')
    if market_CHAN:
        REG_list.append('Market[Market]')
    process_dax_queries(BrandOrTopB, hierarchy_levels, time_filter=True, area="REGION", row_list=REG_list, brackets=brackets,areamarket=True)


if f'{customareas}' in areas:
    CUST_list = []
    if regions_CUST:
        CUST_list.append('Market[Region]')
    if channels_CUST:
        CUST_list.append('Market[Channel]')
    if market_CUST:
        CUST_list.append('Market[Market]')
    process_dax_queries(BrandOrTopB, hierarchy_levels, time_filter=True, area=customareas, row_list=CUST_list, brackets=brackets,areamarket=True)

Query executed successfully for Segment | ['Market[Area]'].
Query executed successfully for Segment | ['Market[Area]'].
Query executed successfully for Segment | ['Market[Area]'].
Query executed successfully for Category | ['Market[Area]'].
Query executed successfully for Sector | ['Market[Area]'].
Query executed successfully for Sector | ['Market[Area]'].
Saved DataFrames for ['Market[Area]'] to c:\Users\aleaa\Documents\Slide-Automate\PPA slide duplicate\PPA NewEX\modified_area_base_price_bracket.pkl.
Query executed successfully for Segment | ['Market[Area]'].
Query executed successfully for Sector | ['Market[Area]'].
Query executed successfully for Sector | ['Market[Area]'].
Query executed successfully for Segment | ['Market[Area]'].
Query executed successfully for Segment | ['Market[Area]'].
Query executed successfully for Category | ['Market[Area]'].
Saved DataFrames for ['Market[Area]'] to c:\Users\aleaa\Documents\Slide-Automate\PPA slide duplicate\PPA NewEX\modified_area_pack_siz

########### Total

In [ ]:
def execute_dax_query(BrandorMaunf, entity_name, hierby, filter_p12m, area, bracket, entity_type=None, row_field=None,areamarket=True):
    outputdic = {}

    time_filter = """
        FILTER(
            VALUES('Time Logic'[Time Period]), 
            'Time Logic'[Time Period] = "P12M"
        )
    """ if filter_p12m else ""

    key = f"{entity_name}"
    filters = ", ".join(filter(None, [time_filter]))
    if areamarket is True and area:
        areafilter = f"""FILTER('Market', 'Market'[Area] = "{area}"),"""
    else:
        areafilter = ""
    areafilter = ", ".join(filter(None, [areafilter]))
# Convert row_field list into DISTINCT(SELECTCOLUMNS(...)) parts
    crossjoin_parts = []

    if isinstance(row_field, list):
        for field in row_field:
            table, col = field.split("[")
            col = col.rstrip("]")
            crossjoin_parts.append(
                f'DISTINCT(SELECTCOLUMNS({table.strip()}, "{col.strip()}", {table.strip()}[{col.strip()}]))'
            )
    else:
        table, col = row_field.split("[")
        col = col.rstrip("]")
        crossjoin_parts.append(
            f'DISTINCT(SELECTCOLUMNS({table.strip()}, "{col.strip()}", {table.strip()}[{col.strip()}]))'
        )

    crossjoin_expr = ",\n                ".join(crossjoin_parts)
    row_fields_expr = ", ".join(
    f"[{field.split('[')[1].rstrip(']')}]"
    for field in row_field
    ) if isinstance(row_field, list) else f"[{row_field.split('[')[1].rstrip(']')}]"

    # print(row_fields_expr)
    table_name, column_name = bracket.split("[")
    table_name = table_name.strip()
    column_name = column_name.rstrip("]").strip()

    # Build summarize_args as a string with proper quoting
    if table_name == 'Base Price Bracket':
        column_name="Base Price Bracket"
        

    else:
        rowlst = f"{table_name},{table_name}[{column_name}]"

    # print(crossjoin_expr)
    crossjoin_expr_list = [crossjoin_expr]
    crossjoin_expr_list=crossjoin_expr_list[0]
    row_fields_exprlist=[row_fields_expr]
    row_fields_expr_list = [field.strip() for field in row_fields_expr.split(",")]
    row_fields_expr_first = row_fields_expr_list[0]
    dax_query = f"""
    EVALUATE
    CALCULATETABLE(
        ADDCOLUMNS(
            SUMMARIZE(
                CROSSJOIN(

                    DISTINCT(SELECTCOLUMNS('{table_name}', "{column_name}", '{table_name}'[{column_name}])),
                    {crossjoin_expr}
                    
                ),
                [{column_name}],{row_fields_expr}
            ),
            "Value Share", COALESCE([Value Share], 0),
            "WoB %", COALESCE([WoB %], 0),
            "Value Sales IYA", COALESCE([Value Sales IYA], 0),
            "Relative Price", COALESCE([Relative Price], 0)
        ),
        {time_filter},
        {areafilter} 
        FILTER(Products,Products[{hierby}] = "{entity_type}"),
        FILTER('Scope', 'Scope'[Scope] = "{hierby}")
    )
    """
    
    maintotal_dax_query = f"""
        EVALUATE
            CALCULATETABLE(
                ADDCOLUMNS(
                    SUMMARIZE(
                    CROSSJOIN(
                        DISTINCT(SELECTCOLUMNS('{table_name}', "{column_name}", '{table_name}'[{column_name}])),
                        {crossjoin_expr_list}
                        
                    ),
                    [{column_name}],{row_fields_expr_first}
                ),
                    "Value Share", COALESCE([Value Share], 0),
                    "WoB %", COALESCE([WoB %], 0),
                    "Value Sales IYA", COALESCE([Value Sales IYA], 0),
                    "Relative Price", COALESCE([Relative Price], 0)
                ),
                {time_filter},
                {areafilter}     
                FILTER(Products,Products[{hierby}] = "{entity_type}"),
                FILTER('Scope', 'Scope'[Scope] = "Category")
            )
    """

    try:
        with adodbapi.connect(conn_str) as conn, conn.cursor() as cursor:
            cursor.execute(dax_query)
            columns = [desc[0] for desc in cursor.description]
            data = cursor.fetchall()
            
        with adodbapi.connect(conn_str) as conn, conn.cursor() as cursor:
            cursor.execute(maintotal_dax_query)
            maintotal_columns = [desc[0] for desc in cursor.description]
            maintotal_data = cursor.fetchall()

        df = pd.DataFrame(data, columns=columns)
        df.columns = df.columns.str.replace(r'.*\[|\]', '', regex=True)
        df = df.loc[~(df.select_dtypes(include='number') == 0).all(axis=1)]

        maintotal_df = pd.DataFrame(maintotal_data, columns=maintotal_columns)
        maintotal_df.columns = maintotal_df.columns.str.replace(r'.*\[|\]', '', regex=True)
        # print(maintotal_df)
        maintotal_df = maintotal_df.loc[~(maintotal_df.select_dtypes(include='number') == 0).all(axis=1)]
        if maintotal_df.shape[1] > 1:
                maintotal_df.iloc[:, 1] = maintotal_df.iloc[:, 1].astype(str) + " total"


        if not maintotal_df.empty:
            # Ensure the total dataframe has the same columns as df
            for col in df.columns:
                if col not in maintotal_df.columns:
                    maintotal_df[col] = pd.NA  # or np.nan if you prefer numeric NaNs

            # Reorder columns to match df
            maintotal_df = maintotal_df[df.columns]
            frames = [df_ for df_ in [maintotal_df, df] if not df_.empty and not df_.isna().all(axis=None)]
            df_with_totals = pd.concat(frames, ignore_index=True)

            # Concatenate totals on top of the main dataframe


            outputdic[key] = df_with_totals
        print(f"Query executed successfully for {hierby} | {row_field}.")
    except adodbapi.DatabaseError as db_error:
        print(f"Database error for {hierby} {entity_type} | {row_field}: {db_error}")

    return outputdic


def process_dax_queries(BrandorManuf, hierarchy_levels, time_filter, area, row_list, brackets,areamarket):
    with ThreadPoolExecutor(max_workers=8) as executor:
        for bracket in brackets:
            dfs_results = {}  # ← Reset per bracket
            futures = []      # ← Reset per bracket

            for hierby, hier_values in hierarchy_levels:
                for value in hier_values:
                    futures.append(executor.submit(
                        execute_dax_query,
                        BrandorManuf, value, hierby, time_filter, area, bracket,
                        entity_type=value, row_field=row_list
                    ))

            for future in as_completed(futures):
                try:
                    result = future.result()
                    dfs_results.update(result)
                except Exception as e:
                    print(f"Error processing query: {e}")

            # suffix = row_list[0].split('[')[-1].rstrip(']')
            column_name = bracket.split("[")[-1].rstrip("]")
            if area == "RETAILER" and areamarket==True:
                filename = f"modified_catretailer_{column_name}.pkl".replace(" ", "_").replace("\xa0", "_").lower()
            elif area == "CHANNEL" and areamarket==True:
                filename = f"modified_catchannels_{column_name}.pkl".replace(" ", "_").replace("\xa0", "_").lower()
            elif area == "REGION" and areamarket==True:
                filename = f"modified_catregion_{column_name}.pkl".replace(" ", "_").replace("\xa0", "_").lower()
            elif area == customareas and areamarket==True:
                filename = f"modified_catcust_{column_name}.pkl".replace(" ", "_").replace("\xa0", "_").lower()
            else:
                filename = f"modified_cat_Area_{column_name}.pkl".replace(" ", "_").replace("\xa0", "_").lower()

            output_file = f"{path}\\{filename}"
            with open(output_file, "wb") as f:
                pd.to_pickle(dfs_results, f)
            print(f"Saved DataFrames for {row_list} to {output_file}.")


area_list = []
area_list.append('Market[Area]')
process_dax_queries(BrandOrTopB, hierarchy_levels, time_filter=True, area="", row_list=area_list, brackets=brackets,areamarket=False)
    
if "RETAILER" in areas:
    RET_list = []
    if regions_RET:
        RET_list.append('Market[Region]')
    if channels_RET:
        RET_list.append('Market[Channel]')
    if market_RET:
        RET_list.append('Market[Market]')
    process_dax_queries(BrandOrTopB, hierarchy_levels, time_filter=True, area="RETAILER", row_list=RET_list, brackets=brackets,areamarket=True)

if "CHANNEL" in areas:
    CHA_list = []
    if regions_CHAN:
        CHA_list.append('Market[Region]')
    if channels_CHAN:
        CHA_list.append('Market[Channel]')
    if market_CHAN:
        CHA_list.append('Market[Market]')
    process_dax_queries(BrandOrTopB, hierarchy_levels, time_filter=True, area="CHANNEL", row_list=CHA_list, brackets=brackets,areamarket=True)

if "REGION" in areas:
    REG_list = []
    if regions_CHAN:
        REG_list.append('Market[Region]')
    if channels_CHAN:
        REG_list.append('Market[Channel]')
    if market_CHAN:
        REG_list.append('Market[Market]')
    process_dax_queries(BrandOrTopB, hierarchy_levels, time_filter=True, area="REGION", row_list=REG_list, brackets=brackets,areamarket=True)

if f'{customareas}' in areas:
    CUST_list = []
    if regions_CUST:
        CUST_list.append('Market[Region]')
    if channels_CUST:
        CUST_list.append('Market[Channel]')
    if market_CUST:
        CUST_list.append('Market[Market]')
    process_dax_queries(BrandOrTopB, hierarchy_levels, time_filter=True, area=customareas, row_list=CUST_list, brackets=brackets,areamarket=True)

Query executed successfully for Segment | ['Market[Area]'].
Query executed successfully for Segment | ['Market[Area]'].
Query executed successfully for Sector | ['Market[Area]'].
Query executed successfully for Segment | ['Market[Area]'].
Query executed successfully for Category | ['Market[Area]'].
Query executed successfully for Sector | ['Market[Area]'].
Saved DataFrames for ['Market[Area]'] to c:\Users\aleaa\Documents\Slide-Automate\PPA slide duplicate\PPA NewEX\modified_cat_area_base_price_bracket.pkl.
Query executed successfully for Sector | ['Market[Area]'].
Query executed successfully for Category | ['Market[Area]'].
Query executed successfully for Segment | ['Market[Area]'].
Query executed successfully for Segment | ['Market[Area]'].
Query executed successfully for Segment | ['Market[Area]'].
Query executed successfully for Sector | ['Market[Area]'].
Saved DataFrames for ['Market[Area]'] to c:\Users\aleaa\Documents\Slide-Automate\PPA slide duplicate\PPA NewEX\modified_cat_area_

In [154]:
end_time = time.time()
elapsed_time = end_time - start_time
print(f"Script started at: {time.ctime(start_time)}")
print(f"Script ended at: {time.ctime(end_time)}")
print(f"Elapsed time: {elapsed_time:.2f} seconds")

Script started at: Wed Jul  9 12:50:26 2025
Script ended at: Wed Jul  9 12:52:36 2025
Elapsed time: 130.16 seconds


In [155]:
loaded_data = {}
datasets_path =os.getcwd()+"\\PPA NewEX\\"
datasets = os.listdir(datasets_path)
for d in datasets:
    with open(datasets_path+d, 'rb') as handle:
        globals()[d.split('.')[0]] = pd.read_pickle(handle)